In [1]:
%config IPCompleter.greedy=True
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [3]:
cifar100 = tf.keras.datasets.cifar100

VGG19 Model <br>

<ul>
block 1<Br>
conv 3*3, 64<br>
conv 3*3, 64<br>
maxpool 2 
</ul>

<ul>
block 2<Br>
conv 3*3, 128<br>
conv 3*3, 128<br>
maxpool 2 
</ul>

<ul>
block 3<Br>
conv 3*3, 256<br>
conv 3*3, 256<br>
conv 3*3, 256<br>
maxpool 2
</ul>

<ul>
block 4<Br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
maxpool 2 
</ul>


<ul>
block 5<Br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
conv 3*3, 512<br>
maxpool 2 
</ul>

<ul>
block 6 Fully Connect<Br>
Fully Connect Network 4096<br>
Fully Connect Network 4096<br>
Fully Connect Netowrk 100<br>
</ul>


In [35]:
class ConvLayer2(tf.keras.Model):
    def __init__(self, filters, size):
        super(ConvLayer2, self).__init__()
        self.conv1 = layers.Conv2D(filters, [size,size], activation='relu')
        self.conv2 = layers.Conv2D(filters, [size,size], activation='relu')
        self.pool = layers.MaxPool2D()
        
    def call(self, x):
        model = self.conv1(x)
        model = self.conv2(model)
        model = self.pool(model)
        return model

class ConvLayer3(tf.keras.Model):
    def __init__(self, filters, size):
        super(ConvLayer3, self).__init__()
        self.conv1 = layers.Conv2D(filters, [size,size], activation='relu')
        self.conv2 = layers.Conv2D(filters, [size,size], activation='relu')
        self.conv3 = layers.Conv2D(filters, [size,size], activation='relu')
        self.pool = layers.MaxPool2D()
        
    def call(self, x):
        model = self.conv1(x)
        model = self.conv2(model)
        model = self.conv3(model)
        model = self.pool(model)
        return model

class ConvLayer4(tf.keras.Model):
    def __init__(self, filters, size):
        super(ConvLayer4, self).__init__()
        self.conv1 = layers.Conv2D(filters, [size,size], activation='relu')
        self.conv2 = layers.Conv2D(filters, [size,size], activation='relu')
        self.conv3 = layers.Conv2D(filters, [size,size], activation='relu')
        self.conv4 = layers.Conv2D(filters, [size,size], activation='relu')
        self.pool = layers.MaxPool2D()
        
    def call(self, x):
        model = self.conv1(x)
        model = self.conv2(model)
        model = self.conv3(model)
        model = self.conv4(model)
        model = self.pool(model)
        return model

class VGG19(tf.keras.Model):
    def __init__(self, img_class):
        super(VGG19, self).__init__()
        self.conv_block1 = ConvLayer2(64, 3)
        self.conv_block2 = ConvLayer2(128, 3)
        self.conv_block3 = ConvLayer3(256, 3)
        self.conv_block4 = ConvLayer4(512, 3)
        self.conv_block5 = ConvLayer4(512, 3)
        self.conv_block6 = ConvLayer4(512, 3)
        self.flat = layers.Flatten()
        self.fcn1 = layers.Dense(4096, activation='relu')
        self.fcn2 = layers.Dense(4096, activation='relu')
        self.fcn3 = layers.Dense(img_class, activation='relu')
        
    def call(self, x):
        model = self.conv_block1(x)
        model = self.conv_block2(model)
        model = self.conv_block3(model)
        model = self.conv_block4(model)
#         model = self.conv_block5(model)
#         model = self.conv_block6(model)
        model = self.flat(model)
        model = self.fcn1(model)
        model = self.fcn2(model)
        model = self.fcn3(model)
        return model

In [36]:
model = VGG19(1000)
print(model)
x = tf.random_normal([1,224,224,3])
print(model(x).shape)

(1, 1000)


In [37]:
tf.keras.datasets.

[<tf.Variable 'vg_g19_14/conv_layer2_28/conv2d_238/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
array([[[[ 9.28332657e-03, -4.25630473e-02, -6.28011674e-02, ...,
          -9.13968161e-02, -3.64275947e-02, -3.35975736e-02],
         [ 1.73789039e-02, -5.42941578e-02,  9.37960148e-02, ...,
           9.81325358e-02,  9.31473821e-02,  1.84754282e-03],
         [-2.89084837e-02,  6.14336580e-02, -2.78056189e-02, ...,
          -4.16819751e-02,  8.86196494e-02,  1.14200860e-02]],

        [[-6.17057085e-04, -2.68532485e-02, -2.46608853e-02, ...,
           1.37377605e-02, -3.24673131e-02,  3.04396003e-02],
         [-1.46498606e-02,  8.69191438e-02, -4.83708940e-02, ...,
           8.18420649e-02,  3.99081558e-02, -8.26939940e-05],
         [ 8.48620832e-02, -2.45348588e-02, -1.82947665e-02, ...,
           7.27318227e-04,  5.98863959e-02,  8.64136219e-03]],

        [[ 1.34801194e-02, -9.57687199e-03, -4.57724184e-02, ...,
           2.77788341e-02,  5.04840165e-03, -4.29290570e-02

In [ ]:
optimizer = tf.train.AdamOptimizer()
epoch=2
loss_history = []
acc_history = []

for _ in range(epoch):
    for ( batch, (images, labels)) in enumerate(dataset.take(6000)):
        accuracy = tfe.metrics.Accuracy('accuracy', dtype=tf.float32)
        with tf.GradientTape() as tape:
            logits = model(images)
            loss_value = tf.losses.sparse_softmax_cross_entropy(labels, logits)
            accuracy(tf.argmax(logits, axis=1, output_type=tf.int64),
                    tf.cast(labels, tf.int64))

        loss_history.append(loss_value.numpy())
        acc_history.append(accuracy.result())
        grads = tape.gradient(loss_value, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables), 
                                 global_step=tf.train.get_or_create_global_step())
        if batch % 20== 0 :
            print("Loss at step {:03d}: {:.3f}, Accu : {:.3f}".format(batch, loss_value, accuracy.result()))
